# Pipeline Prototype

In [17]:
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
from models.tf.models import LongShortTermMemory
from models.tf.dataset import StockDataGenerator
from sklearn.model_selection import KFold, GridSearchCV
from joblib import Memory

In [18]:
VERBOSITY = 1

In [19]:
feature_combos = { 
    'set1': {}, 
    'set2':
    { 
        'EMA': {}, '%B': {}, 'CCI': {}, 
        'RSI': {}, 'VIX': {}, 'MIDPRICE': {}, 
        'MAMA': {}, 'ROCR100': {}
    },
    'set3':
    {
        'EMA': {}, '%B': {'timeperiod': 15}, 'CCI': {}, 
        'RSI': {}, 'VIX': {}, 'MIDPRICE': {'timeperiod': 15},
        'MAMA': {}, 'ROCR100': {}, 'AROONOSC': {}
    }
}